In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch
import torch.nn as nn               # LinearRegression 기능의 클래스 Linear
import torch.optim as optim         # 최적화 (경사하강법)
import torch.nn.functional as F     # 손실함수 로딩


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:

db_name = 'Fashion-MNIST'
X, y = fetch_openml(name=db_name, parser='auto', as_frame=False, return_X_y=True)


In [4]:
Xt_train = torch.from_numpy(X[:60000]).type(torch.float32)
yt_train = torch.from_numpy(y[:60000].astype(np.uint8)).type(torch.long)
Xt_test = torch.from_numpy(X[60000:]).type(torch.float32)
yt_test = torch.from_numpy(y[60000:].astype(np.uint8)).type(torch.long)

# assign to cuda
Xt_train, yt_train = Xt_train.to(device), yt_train.to(device)
Xt_test, yt_test = Xt_test.to(device), yt_test.to(device)

yt_train = F.one_hot(yt_train, num_classes=10).type(torch.float32)
yt_test = F.one_hot(yt_test, num_classes=10).type(torch.float32)

In [5]:
class model(nn.Module):
    """다중 분류 모델, 
    활성함수는 별도의 loss function 
    에 내장된 cross_entropy를 사용"""
    def __init__(self, device):
        super(model, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(784, 256, device=device),
            nn.ReLU(),
            nn.Linear(256, 128, device=device),
            nn.ReLU(),
            nn.Linear(128, 10, device=device),
        )

    def forward(self, x):
        x = self.seq(x)
        return x
    


In [6]:
def accuracy_cross_entropy(y_pred, y_true, y_test_pred, y_test_true):
    trainscore =  torch.mean((torch.argmax(y_pred, dim=1) == torch.argmax(y_true, dim=1)).type(torch.float32))
    valscore =  torch.mean((torch.argmax(y_test_pred, dim=1) == torch.argmax(y_test_true, dim=1)).type(torch.float32))
    return trainscore, valscore


def train(model, X_train, y_train, X_test, y_test, criterion, optimizer, epochs=10):
    model.train()
    totalresult = [[],[]]
    optimizer = optimizer(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', patience=5, verbose=True, eps=1e-8)
    # try:
    #     if not torch.cuda.is_available():
    #         import intel_extension_for_pytorch as ipex
    #         model, optimizer = ipex.optimize(model, optimizer = optimizer)
    # except:
    #     print("Intel Ipex Disabled")
    
    for epoch in range(epochs+1):
        y_pred = model(X_train)
        y_test_pred  = model(X_test)
        
        loss = criterion(y_pred, y_train)
        val_loss = criterion(y_test_pred, y_test)
        
        optimizer.zero_grad()
        loss.backward()
        scheduler.step(val_loss)
        optimizer.step()
        
        result = accuracy_cross_entropy(y_pred, y_train, y_test_pred, y_test)
        totalresult[0].append(result[0])
        totalresult[1].append(result[1])
        
        
        print(f'Epoch {epoch}, Loss: {loss.item()},\n'
              f'Acurracy : (1) train {result[0]:.3f}, (2) test {result[1]:.3f}')
        
        if scheduler.num_bad_epochs > scheduler.patience:
            print(f'Early stopping at epoch {epoch} ... ')
            break
    model.eval()

In [7]:
cloth = model(device)
criteria = F.cross_entropy
optimizer = optim.Adam
epochs = 100
train(
    cloth, 
    Xt_train, 
    yt_train, 
    Xt_test, 
    yt_test, 
    criteria, 
    optimizer, 
    epochs=epochs
)


/home/workplace/miniconda3/envs/EXAM_DL2/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 0, Loss: 18.63163185119629,
Acurracy : (1) train 0.129, (2) test 0.128
Epoch 1, Loss: 22.389232635498047,
Acurracy : (1) train 0.178, (2) test 0.184
Epoch 2, Loss: 32.15156555175781,
Acurracy : (1) train 0.206, (2) test 0.211
Epoch 3, Loss: 16.61124610900879,
Acurracy : (1) train 0.217, (2) test 0.220
Epoch 4, Loss: 12.173739433288574,
Acurracy : (1) train 0.259, (2) test 0.260
Epoch 5, Loss: 8.669458389282227,
Acurracy : (1) train 0.368, (2) test 0.365
Epoch 6, Loss: 6.756321430206299,
Acurracy : (1) train 0.428, (2) test 0.422
Epoch 7, Loss: 4.763392925262451,
Acurracy : (1) train 0.434, (2) test 0.432
Epoch 8, Loss: 3.3811144828796387,
Acurracy : (1) train 0.505, (2) test 0.498
Epoch 9, Loss: 2.8291521072387695,
Acurracy : (1) train 0.525, (2) test 0.521
Epoch 10, Loss: 2.6364264488220215,
Acurracy : (1) train 0.537, (2) test 0.536
Epoch 11, Loss: 2.381291151046753,
Acurracy : (1) train 0.541, (2) test 0.538
Epoch 12, Loss: 1.534411072731018,
Acurracy : (1) train 0.592, (2) te

In [8]:
for x in cloth.parameters():
    print(x.shape)

torch.Size([256, 784])
torch.Size([256])
torch.Size([128, 256])
torch.Size([128])
torch.Size([10, 128])
torch.Size([10])
